In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utility import *
from scipy.sparse.linalg import eigs
from scipy import sparse

basis = 'sto-3g'

## Function to compute number of measurements needed for a given molecule

In [25]:
def createHam(h, n_qubits = None):

    I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

    # Build matrix representiation of the Hamiltonian H
    if(n_qubits is None):
        n_qubits = openfermion.count_qubits(h)
        #print(n_qubits)
    
    h_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
    for term, term_coeff in h.terms.items(): # Iterate over pauli-words of H
        term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']

        # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
        pw_matrix = np.identity(1)
        for i in range(n_qubits):
            if i not in term:        pw_matrix = np.kron(pw_matrix, I)
            else:
                if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
                elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
                else:                pw_matrix = np.kron(pw_matrix, Z)
        h_matrix += pw_matrix * term_coeff

    return h_matrix

def getSigma2(mname, bondlength=1.0):
    
    h = get_qubit_hamiltonian(mol=mname, geometry=bondlength, basis='sto3g', qubit_transf='jw')
    n_qubits = openfermion.count_qubits(h)
    print('Number of qubits needed:',n_qubits)
    
    h_matrix = createHam(h)
    print('matrix created')
    
    #eigvals, eigvec = np.linalg.eigh(h_matrix)    
    #print('E:',eigvals)
        
    h_sparse = sparse.csr_matrix(h_matrix)
    print('sparse created')
    
    eigvals, eigvec = eigs(h_sparse,k = 1,which='SR', tol=1.0e-6)
    #print(eigvals)
    #print(eigvec)
    psi = eigvec[:,0]
    
    #print('Psi:',psi)    
    #print(np.dot(np.dot(psi.conj(),h_matrix),psi))
    
    
    h_matrix2 = np.dot(h_matrix,h_matrix)
    fsig2 = np.dot(np.dot(psi.conj(),h_matrix2),psi) - np.dot(np.dot(psi.conj(),h_matrix),psi)**2
    
    #Do qubit-wise-commuting partitioning
    qwc_list = get_qwc_group(h)    
    sig2s_qwc = []
    for qwc in qwc_list:
        #print('QWC Fragment: ',qwc)
        
        #Do unitary rotation for z-gate measurement restriction
        uz = get_zform_unitary(qwc)    
        allz = remove_complex(uz * qwc * uz)
        
        lH = createHam(allz, n_qubits = n_qubits)
        #print('QWC Fragment shape:',lH.shape)
        lH2 = np.dot(lH,lH)
        lsig2 = np.dot(np.dot(psi.conj(),lH2),psi) - np.dot(np.dot(psi.conj(),lH),psi)**2
        lsig2 = np.abs(lsig2)
        sig2s_qwc.append(lsig2**0.5)
    
    print('\n')
    #Do fully commuting
    comm_groups = get_commuting_group(h)
    
    sig2s_fc = []
    for hn in comm_groups.values():
        #print('FC Fragment:',hn)
        
        #Create QWC from FC
        uqwc = get_qwc_unitary(hn)
        qwc = remove_complex(uqwc * hn * uqwc)
        
        #Do unitary rotation for z-gate measurement restriction
        uz = get_zform_unitary(qwc)    
        allz = remove_complex(uz * qwc * uz)
        
        lH = createHam(allz, n_qubits = n_qubits)
        lH2 = np.dot(lH,lH)
        lsig2 = np.dot(np.dot(psi.conj(),lH2),psi) - np.dot(np.dot(psi.conj(),lH),psi)**2
        lsig2 = lsig2.real
        sig2s_fc.append(lsig2**0.5)
    
    return sig2s_qwc, sig2s_fc, fsig2
    

In [26]:
def getN_T(etols, sig2s):    
    return (np.sum(sig2s)/etols)**2

In [27]:
sig2s_qwc, sig2s_fc, fsig2 = getSigma2('h2',0.741)

Number of qubits needed: 4
matrix created




In [28]:
print('\nResults\n')
print(f'QWC Sigma Spectrum ({len(sig2s_qwc)}): {sig2s_qwc}')
print('QWC N_T:',getN_T(1.0e-3,sig2s_qwc))
print('\n')
print(f'FC Sigma Spectrum: ({len(sig2s_fc)}): {sig2s_fc}')
print('FC N_T:',getN_T(1.0e-3,sig2s_fc))
print('\n')
print('Full hamiltonian Sigma: ',fsig2)
print('N_T:',getN_T(1.0e-3,[fsig2]))


Results

QWC Sigma Spectrum (5): [1.1406325468715177e-09, 5.607558368760129e-10, 5.607558368760129e-10, 5.607558368760129e-10, 0.17665799649573852]
QWC N_T: 31208.048723264103


FC Sigma Spectrum: (2): [0.004806455699775473, 0.17665799649573885]
FC N_T: 32929.3474106181


Full hamiltonian Sigma:  (-8.881784197001252e-16+1.5240520571051535e-17j)
N_T: (7.886286317537351e-25-2.7072602952407606e-26j)


In [29]:
sig2s_qwc, sig2s_fc, fsig2 = getSigma2('h4',25)

Number of qubits needed: 8
matrix created




In [30]:
print('\nResults\n')
print(f'QWC Sigma Spectrum ({len(sig2s_qwc)}): {sig2s_qwc}')
print('QWC N_T:',getN_T(1.0e-3,sig2s_qwc))
print('\n')
print(f'FC Sigma Spectrum: ({len(sig2s_fc)}): {sig2s_fc}')
print('FC N_T:',getN_T(1.0e-3,sig2s_fc))
print('\n')
print('Full hamiltonian Sigma: ',fsig2)
print('N_T:',getN_T(1.0e-3,[fsig2]))


Results

QWC Sigma Spectrum (37): [0.17007661475031433, 7.054960604588239e-08, 7.054960604588239e-08, 7.054960604588239e-08, 0.09449681018401673, 0.009359780361883407, 0.09484555001014684, 0.009359780362302339, 0.005080087172574179, 0.014539669435145449, 0.014539669434571547, 0.005080087174410911, 0.0050800871714330766, 0.005080087174410911, 0.005080087170437591, 0.014539669435312081, 0.01453966943559647, 0.01453966943559647, 0.014539669435312081, 0.005080087170437591, 0.0050800871714330766, 0.014539669434571547, 0.005080087172574179, 0.014539669435145449, 0.0001681742749759163, 0.00016817426528461862, 0.00016817426528461862, 0.009191606101965092, 0.00016817427439827436, 0.009191606101846052, 0.009191606101268252, 0.009191606101965092, 0.046510985780851215, 5.021158961731636e-08, 5.021158961731636e-08, 5.021158961731636e-08, 0.09302446470237796]
QWC N_T: 507045.8535562652


FC Sigma Spectrum: (4): [0.010388498464063197, 0.044062001581214046, 0.01461114058787768, 0.34635984384585766]
F

In [ ]:
sig2s_qwc, sig2s_fc, fsig2 = getSigma2('lih',1.25)

Number of qubits needed: 12
matrix created




In [ ]:
print('\nResults\n')
print(f'QWC Sigma Spectrum ({len(sig2s_qwc)}): {sig2s_qwc}')
print('QWC N_T:',getN_T(1.0e-3,sig2s_qwc))
print('\n')
print(f'FC Sigma Spectrum: ({len(sig2s_fc)}): {sig2s_fc}')
print('FC N_T:',getN_T(1.0e-3,sig2s_fc))
print('\n')
print('Full hamiltonian Sigma: ',fsig2)
print('N_T:',getN_T(1.0e-3,[fsig2]))